# Main results: presentation of the model and the proof of its validity
Tuning curves prediction and position decoding from firing rates.

- a. Schematic of the model
- b. Examples of tunign curves pre and post alignment
- c. Quantify the similarity of the tuning curves after alignment
- d. Show the performance of the decoders built on one session and used across

In [1]:
import os
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.sys.path.append(parent_directory)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from tools.data_formatting import get_smoothed_moving_all_data, get_common_indexes_2recordings

## Fig.2.b 

In [7]:
# Load data
animal = 'm120'
fov = 'fov1'
experiment1 = 'fam1novfam1'
run1 = 'fam1'
experiment2 = 'fam1novfam1'
run2 = 'fam1r2'
n_points = 360

firing_rates_run1, time_run1, phi_run1, cells_run1, tuning_curves1, phi_bins_run1 = get_smoothed_moving_all_data(animal, fov, experiment1, run1, n_points=n_points)
firing_rates_run2, time_run2, phi_run2, cells_run2, tuning_curves2, phi_bins_run2 = get_smoothed_moving_all_data(animal, fov, experiment2, run2, n_points=n_points)

common_cells_run1, common_cells_run2, ordered_cells_run1, ordered_cells_run2 = get_common_indexes_2recordings(cells_run1, cells_run2)
tuning_curves1 = tuning_curves1[:, common_cells_run1][:, ordered_cells_run1]
tuning_curves2 = tuning_curves2[:, common_cells_run2][:, ordered_cells_run2]
cells_run1 = cells_run1[common_cells_run1][ordered_cells_run1]
cells_run2 = cells_run2[common_cells_run2][ordered_cells_run2]

print(f'Runs: {experiment1} {run1} - {experiment2} {run2}\nShape tuning_curves: {tuning_curves1.shape} - {tuning_curves2.shape}')

Runs: fam1novfam1 fam1 - fam1novfam1 fam1r2
Shape tuning_curves: (360, 186) - (360, 186)


In [5]:
# Align the tuning curves and project in other space
pca = PCA(n_components=20)
pca_tuning_curves1 = pca.fit_transform(tuning_curves1)
components_run1 = pca.components_
pca_tuning_curves2 = pca.fit_transform(tuning_curves2)
components_run2 = pca.components_



(360, 186)